In [1]:
import os

In [2]:
%pwd

'n:\\Users\\Farshu\\Projects\\Data-Science-Project-with-MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'n:\\Users\\Farshu\\Projects\\Data-Science-Project-with-MLFlow'

In [7]:
import pandas as pd

In [8]:
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [10]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [11]:
data.shape

(1599, 12)

In [12]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [ ]:

from dataclasses import dataclass
from pathlib import Path


# Defining a dataclass for Data Validation Configuration
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path         # Root directory for data validation artifacts
    STATUS_FILE: str       # File to store validation status
    unzip_data_dir: Path   # Directory where unzipped data is stored
    all_schema: dict       # Dictionary containing schema details

In [14]:
# Importing necessary modules for configuration management
from mlProject.constants import *  # Importing constants like file paths
from mlProject.utils.common import read_yaml, create_directories  # Utility functions

In [15]:
# Configuration Manager Class to handle configurations
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,  # Path to config.yaml
        params_filepath = PARAMS_FILE_PATH,  # Path to params.yaml
        schema_filepath = SCHEMA_FILE_PATH   # Path to schema.yaml
    ):

        self.config = read_yaml(config_filepath)  # Reading main config file
        self.params = read_yaml(params_filepath)  # Reading parameters file
        self.schema = read_yaml(schema_filepath)  # Reading schema file

        create_directories([self.config.artifacts_root])  # Creating required directories

    # Method to get data validation configuration
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation  # Extracting data validation config
        schema = self.schema.COLUMNS  # Extracting schema details

        create_directories([config.root_dir])  # Ensuring root directory exists

        # Creating an instance of DataValidationConfig with required parameters
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config  # Returning the configuration object

In [ ]:
import os
from mlProject import logger

In [22]:
# DataValidation class to validate dataset columns and data types
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config  # Store configuration

    # Function to validate all columns and data types in the dataset
    def validate_all_columns(self) -> bool:
        try:
            validation_status = True  # Initialize validation status

            # Load the dataset
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)  # Get dataset columns
            all_schema = self.config.all_schema  # Expected schema columns and types

            # Check if dataset columns match expected schema and validate data types
            for col, expected_dtype in all_schema.items():
                if col not in all_cols:
                    validation_status = False
                    logger.error(f"Missing column: {col}")
                else:
                    actual_dtype = str(data[col].dtype)
                    expected_dtype = expected_dtype.lower()
                    if "int" in expected_dtype and "int" not in actual_dtype:
                        validation_status = False
                        logger.error(f"Column {col} has incorrect type. Expected: {expected_dtype}, Found: {actual_dtype}")
                    elif "float" in expected_dtype and "float" not in actual_dtype:
                        validation_status = False
                        logger.error(f"Column {col} has incorrect type. Expected: {expected_dtype}, Found: {actual_dtype}")

            # Write validation status to file
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status  # Return final validation status
        
        except Exception as e:
            logger.exception("Error during validation")
            raise e  # Raise exception if validation fails

In [23]:
# Run the validation process
try:
    config = ConfigurationManager()  # Create configuration manager instance
    data_validation_config = config.get_data_validation_config()  # Get validation config
    data_validation = DataValidation(config=data_validation_config)  # Create validation instance
    data_validation.validate_all_columns()  # Perform validation
except Exception as e:
    logger.exception(e)
    raise e  # Raise exception if any error occurs

[2025-03-28 12:44:32,459: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-03-28 12:44:32,465: INFO: common: YAML file: params.yaml loaded successfully]
[2025-03-28 12:44:32,477: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-03-28 12:44:32,481: INFO: common: Created directory at: artifacts]
[2025-03-28 12:44:32,485: INFO: common: Created directory at: artifacts/data_validation]
